In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path='/content/gdrive/MyDrive/Datasets/'
movies_df= pd.read_csv(path+'movies.csv', dtype={'movieId': 'int32', 'title': 'str'})
ratings_df= pd.read_csv(path+'ratings.csv',usecols=['userId', 'movieId', 'rating'],
            dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
df = pd.merge(ratings_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
df= df.dropna(axis=0,subset=['title'])

In [ ]:
movie_ratingCount= (df.
     groupby('title')['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'}))
movie_ratingCount.head(10)

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
5,'Tis the Season for Love (2015),1
6,"'burbs, The (1989)",17
7,'night Mother (1986),1
8,(500) Days of Summer (2009),42
9,*batteries not included (1987),7


In [ ]:
movie_with_totalRatingCount= df.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
movie_with_totalRatingCount.head()


,userId,movieId,rating,title,genres,totalRatingCount
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [ ]:
popularity_threshold = 50
query_string = f'totalRatingCount >= {popularity_threshold}'
rating_popular_movie = movie_with_totalRatingCount.query(query_string)
rating_popular_movie.head()

,userId,movieId,rating,title,genres,totalRatingCount
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [ ]:
tfidf= TfidfVectorizer(stop_words='english')
tfidf_matrix= tfidf.fit_transform(movies_df['genres'])
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
indices= pd.Series(movies_df.index,index=movies_df['title']).drop_duplicates()
indices

title
Toy Story (1995)                                0
Jumanji (1995)                                  1
Grumpier Old Men (1995)                         2
Waiting to Exhale (1995)                        3
Father of the Bride Part II (1995)              4
                                             ... 
Black Butler: Book of the Atlantic (2017)    9737
No Game No Life: Zero (2017)                 9738
Flint (2017)                                 9739
Bungo Stray Dogs: Dead Apple (2018)          9740
Andrew Dice Clay: Dice Rules (1991)          9741
Length: 9742, dtype: int64

In [ ]:
def get_recommendation(title,cosine_sim =cosine_sim):
    idx=indices[title]
    sim_scores=enumerate(cosine_sim[idx])
    sim_scores= sorted(sim_scores,key=lambda x: x[1],reverse=True)
    sim_scores= sim_scores[1:11]
    sim_index = [i[0] for i in sim_scores]
    print(movies_df['title'].iloc[sim_index])

get_recommendation('Tangled (2010)')

512                  Beauty and the Beast (1991)
2381                        Fantasia 2000 (1999)
7837                 Dr. Seuss' The Lorax (2012)
812       Aladdin and the King of Thieves (1996)
780                            Cinderella (1950)
7207           Princess and the Frog, The (2009)
7133    Cloudy with a Chance of Meatballs (2009)
7181                   Christmas Carol, A (2009)
6785                   Frosty the Snowman (1969)
6626                            Enchanted (2007)
Name: title, dtype: object
